# Enhancing RAG with Contextual Retrieval

We will use an LLM to generate for each chunk and document a contextual sentence to improve its retrival accuracy and use in hybrid search.

* [Load complex documents dataset](#loading-a-complex-dataset-of-documents)
* [Split the documents into chunks](#split-the-documents-into-chunks)
* [Generate the context sentence](#generate-the-context-sentence)
* [Enrich the chunk embedding vectors with the context](#enrich-the-chunk-embedding-vectors-with-the-context)

### Visual improvements

We will use [rich library](https://github.com/Textualize/rich) to make the output more readable, and supress warning messages.

In [10]:
from rich.console import Console
from rich_theme import custom_theme

# Create a console with the dark theme
console = Console(theme=custom_theme)

In [11]:
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

## Loading a complex dataset of documents

We will load a complex dataset of scientific documents from Arxiv. Applying naive chunks on such documents will give poor results in RAG applications.

In [12]:
from datasets import load_dataset

dataset = load_dataset("jamescalam/ai-arxiv2", split="train")
console.print(dataset)

Dataset({
    features: ['id', 'title', 'summary', 'source', 'authors', 'categories', 'comment', 'journal_ref', 
'primary_category', 'published', 'updated', 'content', 'references'],
    num_rows: 2673
})

## Split the documents into Chunks

We will use the statistical chunker that we used in a previous notebook.

In [13]:
import os
from semantic_router.encoders import OpenAIEncoder

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

encoder = OpenAIEncoder(name="text-embedding-3-small")

In [14]:
from semantic_chunkers import StatisticalChunker
import logging

logging.disable(logging.CRITICAL)

chunker = StatisticalChunker(
    encoder=encoder,
    min_split_tokens=100,
    max_split_tokens=500,
)

In [15]:
chunks_0 = chunker(docs=[dataset["content"][0]])


In [16]:
first_chunk = ' '.join(chunks_0[0][0].splits)
console.print(first_chunk)

4 2 0 2 n a J 8 ] G L . s c [ 1 v 8 8 0 4 0 . 1 0 4 2 : v i X r a # Mixtral of Experts Albert Q. Jiang, Alexandre 
Sablayrolles, Antoine Roux, Arthur Mensch, Blanche Savary, Chris Bamford, Devendra Singh Chaplot, Diego de las 
Casas, Emma Bou Hanna, Florian Bressand, Gianna Lengyel, Guillaume Bour, Guillaume Lample, LÃ©lio Renard Lavaud, 
Lucile Saulnier, Marie-Anne Lachaux, Pierre Stock, Sandeep Subramanian, Sophia Yang, Szymon Antoniak, Teven Le 
Scao, ThÃ©ophile Gervet, Thibaut Lavril, Thomas Wang, TimothÃ©e Lacroix, William El Sayed Abstract We introduce 
Mixtral 8x7B, a Sparse Mixture of Experts (SMoE) language model. Mixtral has the same architecture as Mistral 7B, 
with the difference that each layer is composed of 8 feedforward blocks (i.e. experts). For every token, at each 
layer, a router network selects two experts to process the current state and combine their outputs. Even though 
each token only sees two experts, the selected experts can be different at each timestep. As a result, each token 
has access to 47B parameters, but only uses 13B active parameters during inference. Mixtral was trained with a 
context size of 32k tokens and it outperforms or matches Llama 2 70B and GPT-3.5 across all evaluated benchmarks. 
In particular, Mixtral vastly outperforms Llama 2 70B on mathematics, code generation, and multilingual benchmarks.
We also provide a model fine- tuned to follow instructions, Mixtral 8x7B â Instruct, that surpasses GPT-3.5 Turbo, 
Claude-2.1, Gemini Pro, and Llama 2 70B â

## Generate the context sentence

We will use Anthropic Claude for the generation of the context. It is one of the best summarization LLM, and it introduced the [Prompt Caching](https://www.anthropic.com/news/prompt-caching) that is great for the generation of the context for many chunks of the same document.

In [17]:
from dotenv import load_dotenv

load_dotenv()

True

In [18]:
import anthropic

client = anthropic.Anthropic(
    # This is the default and can be omitted
    api_key=os.getenv("ANTHROPIC_API_KEY"),
)


In [19]:
DOCUMENT_CONTEXT_PROMPT = """
<document>
{doc_content}
</document>
"""

CHUNK_CONTEXT_PROMPT = """
Here is the chunk we want to situate within the whole document
<chunk>
{chunk_content}
</chunk>

Please give a short succinct context to situate this chunk within the overall document for the purposes of improving search retrieval of the chunk.
Answer only with the succinct context and nothing else.
"""

def situate_context(doc: str, chunk: str) -> str:
    response = client.beta.prompt_caching.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=1024,
        temperature=0.0,
        messages=[
            {
                "role": "user", 
                "content": [
                    {
                        "type": "text",
                        "text": DOCUMENT_CONTEXT_PROMPT.format(doc_content=doc),
                        "cache_control": {"type": "ephemeral"} #we will make use of prompt caching for the full documents
                    },
                    {
                        "type": "text",
                        "text": CHUNK_CONTEXT_PROMPT.format(chunk_content=chunk),
                    }
                ]
            }
        ],
        extra_headers={"anthropic-beta": "prompt-caching-2024-07-31"}
    )
    return response

In [20]:
chunk_context = situate_context(dataset["content"][0], first_chunk)

In [21]:
console.print(chunk_context)

PromptCachingBetaMessage(
    id='msg_013bNfkmotRvkCU7Di1XfS5e',
    content=[
        TextBlock(
            text='This chunk introduces Mixtral 8x7B, a sparse mixture of experts language model that outperforms 
Llama 2 70B and GPT-3.5 on various benchmarks. It also describes the model architecture and the fine-tuned Mixtral 
8x7B - Instruct model.',
            type='text'
        )
    ],
    model='claude-3-haiku-20240307',
    role='assistant',
    stop_reason='end_turn',
    stop_sequence=None,
    type='message',
    usage=PromptCachingBetaUsage(
        cache_creation_input_tokens=12532,
        cache_read_input_tokens=0,
        input_tokens=584,
        output_tokens=73
    )
)

In [22]:
second_chunk = ' '.join(chunks_0[0][1].splits)

In [23]:
second_chunk_context = situate_context(dataset["content"][0], second_chunk)

In [24]:
console.print(second_chunk_context)

PromptCachingBetaMessage(
    id='msg_01A2Gqcmx4jmGKhtwSMHxLBM',
    content=[
        TextBlock(
            text='This chunk introduces Mixtral 8x7B, a sparse mixture of experts language model that outperforms 
Llama 2 70B and GPT-3.5 on most benchmarks. It describes the key architectural details of Mixtral, including its 
use of a sparse mixture-of-experts network, and mentions that the base and instruct models are released under the 
Apache 2.0 license.',
            type='text'
        )
    ],
    model='claude-3-haiku-20240307',
    role='assistant',
    stop_reason='end_turn',
    stop_sequence=None,
    type='message',
    usage=PromptCachingBetaUsage(
        cache_creation_input_tokens=0,
        cache_read_input_tokens=12532,
        input_tokens=290,
        output_tokens=92
    )
)

## Enrich the chunk embedding vectors with the context

We will iterate over all the chunks. This can take some time based on the number of chunks.

In [25]:
arxiv_id = dataset[0]["id"]
refs = list(dataset[0]["references"].values())
doc_text = dataset[0]["content"]
title = dataset[0]["title"]

from tqdm import tqdm

corpus_json = []
for i, chunk in tqdm(enumerate(chunks_0[0]), total=len(chunks_0[0]), desc="Processing chunks"):
    chunk_text = ' '.join(chunk.splits)
    contextualized_text = situate_context(doc_text, chunk_text).content[0].text
    corpus_json.append({
        "id": i,
        "text": f"{chunk_text}\n\n{contextualized_text}",
        "metadata" : {
            "title": title,
            "arxiv_id": arxiv_id,
            "references": refs
        }
    })

Processing chunks: 100%|██████████| 46/46 [11:10<00:00, 14.58s/it]


### Concatenate the generated context to the chunk text

In [62]:
console.print(corpus_json[:2])

[
    {
        'id': 0,
        'text': '4 2 0 2 n a J 8 ] G L . s c [ 1 v 8 8 0 4 0 . 1 0 4 2 : v i X r a # Mixtral of Experts Albert Q. 
Jiang, Alexandre Sablayrolles, Antoine Roux, Arthur Mensch, Blanche Savary, Chris Bamford, Devendra Singh Chaplot, 
Diego de las Casas, Emma Bou Hanna, Florian Bressand, Gianna Lengyel, Guillaume Bour, Guillaume Lample, LÃ©lio 
Renard Lavaud, Lucile Saulnier, Marie-Anne Lachaux, Pierre Stock, Sandeep Subramanian, Sophia Yang, Szymon 
Antoniak, Teven Le Scao, ThÃ©ophile Gervet, Thibaut Lavril, Thomas Wang, TimothÃ©e Lacroix, William El Sayed 
Abstract We introduce Mixtral 8x7B, a Sparse Mixture of Experts (SMoE) language model. Mixtral has the same 
architecture as Mistral 7B, with the difference that each layer is composed of 8 feedforward blocks (i.e. experts).
For every token, at each layer, a router network selects two experts to process the current state and combine their
outputs. Even though each token only sees two experts, the selected experts can be different at each timestep. As a
result, each token has access to 47B parameters, but only uses 13B active parameters during inference. Mixtral was 
trained with a context size of 32k tokens and it outperforms or matches Llama 2 70B and GPT-3.5 across all 
evaluated benchmarks. In particular, Mixtral vastly outperforms Llama 2 70B on mathematics, code generation, and 
multilingual benchmarks. We also provide a model fine- tuned to follow instructions, Mixtral 8x7B â Instruct, that 
surpasses GPT-3.5 Turbo, Claude-2.1, Gemini Pro, and Llama 2 70B â\n\nThis chunk introduces Mixtral 8x7B, a sparse 
mixture of experts language model that outperforms Llama 2 70B and GPT-3.5 on various benchmarks. It also describes
the model architecture and the fine-tuned Mixtral 8x7B - Instruct model.',
        'metadata': {'title': 'Mixtral of Experts', 'arxiv_id': '2401.04088', 'references': ['1905.07830']}
    },
    {
        'id': 1,
        'text': 'chat model on human bench- marks. Both the base and instruct models are released under the Apache 
2.0 license. Code: https://github.com/mistralai/mistral-src Webpage: https://mistral.ai/news/mixtral-of-experts/ # 
Introduction In this paper, we present Mixtral 8x7B, a sparse mixture of experts model (SMoE) with open weights, 
licensed under Apache 2.0. Mixtral outperforms Llama 2 70B and GPT-3.5 on most benchmarks. As it only uses a subset
of its parameters for every token, Mixtral allows faster inference speed at low batch-sizes, and higher throughput 
at large batch-sizes. Mixtral is a sparse mixture-of-experts network. It is a decoder-only model where the 
feedforward block picks from a set of 8 distinct groups of parameters. At every layer, for every token, a router 
network chooses two of these groups (the â\n\nThis chunk introduces Mixtral 8x7B, a sparse mixture of experts 
language model that outperforms Llama 2 70B and GPT-3.5 on most benchmarks. It describes the key architectural 
details of Mixtral, including its use of a sparse mixture-of-experts network, and mentions that the base and 
instruct models are released under the Apache 2.0 license.',
        'metadata': {'title': 'Mixtral of Experts', 'arxiv_id': '2401.04088', 'references': ['1905.07830']}
    }
]

### Saving the corpus_json in a file

We will want to use it in the next notebook.

In [26]:
import json

with open('data/corpus.json', 'w') as f:
    json.dump(corpus_json, f)

